In [1]:
#Libs
import os
import time

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup as BS
from urllib.request import Request, urlopen

import re

In [2]:
pd.set_option("display.max_rows", 2000)
pd.set_option("max_colwidth", 2000)

In [ ]:
# 1. ALL RATINGS

rev_id_clean = []
overall_rating_clean = []
work_life_balance_clean = []
culture_and_values_clean = []
diversity_inclusion_clean = []
career_opportunities_clean = []
compensation_and_benefits_clean = []
senior_management_clean = []
location_clean = []


df_result_final = pd.DataFrame(columns=['review_id', 'pros', 'cons', 'date_position', 'current_former'])
start_time = time.time()
now = time.asctime()
print(now)

for y in range(213,216):
    url='https://www.glassdoor.com/Reviews/Subway-US-Reviews-EI_IE2994.0,6_IL.7,9_IN1_IP'+str(y)+'.htm?sort.sortType=RD&sort.ascending=false&filter.iso3Language=eng&filter.employmentStatus=REGULAR&filter.employmentStatus=PART_TIME'
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(url,headers=hdr)
    page = urlopen(req)
    bs = BS(page, "html.parser")
    
    time.sleep(20)


    revs = list(bs.find_all('li', attrs = {'class':'noBorder empReview cf pb-0 mb-0'}))
    for x in revs:
        revs2 = str(x)[107:130]
        rev_id_clean.append(revs2)

        y = str(x)   
        
        
        if 'authorLocation' not in y:
            location = 'null'
        else:
            location = y.split('authorLocation',1)[1][:30]
        location_clean.append(location)
        
        if 'ratingNumber mr-x' not in y:
            overall_rating = 'null'
        else:
            overall_rating = y.split('ratingNumber mr-x',1)[1][:25]
        overall_rating_clean.append(overall_rating)
        

        if 'Work/Life Balance' not in y:
            work_life_balance = 'null'
        else:
            work_life_balance = y.split('Work/Life Balance',1)[1][:45]
        work_life_balance_clean.append(work_life_balance)


        if 'Culture &amp; Values' not in y:
            culture_and_values = 'null'
        else:
            culture_and_values = y.split('Culture &amp; Values',1)[1][:45]
        culture_and_values_clean.append(culture_and_values)


        if 'Diversity &amp; Inclusion' not in y:
            diversity_inclusion = 'null'
        else:
            diversity_inclusion = y.split('Diversity &amp; Inclusion',1)[1][:45]
        diversity_inclusion_clean.append(diversity_inclusion)


        if 'Career Opportunities' not in y:
            career_opportunities = 'null'
        else:
            career_opportunities = y.split('Career Opportunities',1)[1][:45]
        career_opportunities_clean.append(career_opportunities)


        if 'Compensation and Benefits' not in y:
            compensation_and_benefits = 'null'
        else:
            compensation_and_benefits = y.split('Compensation and Benefits',1)[1][:45]
        compensation_and_benefits_clean.append(compensation_and_benefits)


        if 'Senior Management' not in y:
            senior_management = 'null'
        else:
            senior_management = y.split('Senior Management',1)[1][:45]
        senior_management_clean.append(senior_management)
        
    review_id = bs.find_all('span', attrs = {'data-disp-type':'review'})
    review_id_clean = []
    pros = bs.find_all('span', attrs = {'data-test':'pros'}) 
    pros_clean = []
    cons = bs.find_all('span', attrs = {'data-test':'cons'}) 
    cons_clean = []
    date_position = bs.find_all('span', attrs = {'class':'authorJobTitle middle common__EiReviewDetailsStyle__newGrey'})
    date_position_clean = []
    current_former = bs.find_all('span', attrs = {'class':'pt-xsm pt-md-0 css-1qxtz39 eg4psks0'})
    current_former_clean = []

    for x in review_id:
        y = str(x)
        m_cut = re.search('data-id="(.+?)" data-member', y)
        if m_cut:
            rev = m_cut.group(1)
        review_id_clean.append(rev)

    for x in pros:
        if x == None or x == "":
            y = 'null'
        else:
            y = "'" + str(x.text.replace("\n", " ").replace("\r", " ").replace(";", " "))
        pros_clean.append(y)

    for x in cons:
        if x == None or x == "":
            y = 'null'
        else:
            y = "'" + str(x.text.replace("\n", " ").replace("\r", " ").replace(";", " "))
        cons_clean.append(y)
    
    for x in date_position:
        if x == None or x == "":
            y = 'null'
        else:
            y = str(x.text)
        date_position_clean.append(y)
    
    for x in current_former:
        if x == None or x == "":
            y = 'null'
        else:
            y = str(x.text)
        current_former_clean.append(y)


    dict = {'review_id': review_id_clean, 'pros': pros_clean, 'cons': cons_clean
            , 'date_position': date_position_clean, 'current_former': current_former_clean}
    df_result = pd.DataFrame(dict)
    df_result_final = df_result_final.append(df_result)
    
    
pre_final_df = pd.DataFrame(rev_id_clean, columns = ['rev_id'])
rev_id_final_df = pre_final_df[pre_final_df['rev_id'].str.contains('view_id')]
rev_id_final_df['review_id'] = rev_id_final_df.rev_id.str.extract('(\d+)')

pre_overall_final_df = pd.DataFrame(overall_rating_clean, columns = ['rate'])
overall_rating_df = pre_overall_final_df[pre_overall_final_df['rate'].str.contains('</span><div')]
overall_rating_df['overall_rating'] = overall_rating_df.rate.str.extract('(\d+)')

rev_id_final_df2 = pd.DataFrame(rev_id_final_df['review_id'], columns = ['review_id'])
overall_rating_df2 = pd.DataFrame(overall_rating_df['overall_rating'], columns = ['overall_rating'])
work_life_balance_df = pd.DataFrame(work_life_balance_clean, columns = ['work_life_balance'])
culture_and_values_df = pd.DataFrame(culture_and_values_clean, columns = ['culture_and_values'])
diversity_inclusion_df = pd.DataFrame(diversity_inclusion_clean, columns = ['diversity_inclusion'])
career_opportunities_df = pd.DataFrame(career_opportunities_clean, columns = ['career_opportunities'])
compensation_and_benefits_df = pd.DataFrame(compensation_and_benefits_clean, columns = ['compensation_and_benefits'])
senior_management_df = pd.DataFrame(senior_management_clean, columns = ['senior_management'])

partial_ratings_final_df = pd.concat([rev_id_final_df2,overall_rating_df2,work_life_balance_df,culture_and_values_df
                                      ,diversity_inclusion_df,career_opportunities_df,compensation_and_benefits_df
                                      ,senior_management_df], axis=1)

cols = ['work_life_balance','culture_and_values','diversity_inclusion','career_opportunities'
        ,'compensation_and_benefits','senior_management']

for x in cols:
    partial_ratings_final_df[x][partial_ratings_final_df[x].str.contains('152xdkl','xd4dom')] = '1'
    partial_ratings_final_df[x][partial_ratings_final_df[x].str.contains('53ckbl','18v8tui')] = '2'
    partial_ratings_final_df[x][partial_ratings_final_df[x].str.contains('xzfimx','vl2edp')] = '3'
    partial_ratings_final_df[x][partial_ratings_final_df[x].str.contains('3irkpa','1nuumx7')] = '4'
    partial_ratings_final_df[x][partial_ratings_final_df[x].str.contains('1dhpfkf','s88v13')] = '5'

all_ratings_final_df = partial_ratings_final_df    
    

# 2. LOCATION

pre_final_df = pd.DataFrame(rev_id_clean, columns = ['rev_id'])
rev_id_final_df = pre_final_df[pre_final_df['rev_id'].str.contains('ew_id')]
rev_id_final_df['review_id'] = rev_id_final_df.rev_id.str.extract('(\d+)')
pre_location_df = pd.DataFrame(location_clean, columns = ['location'])
pre_location_df['location'] = pre_location_df.location.str.extract('">(.*?)</span>')

rev_id_final_df2 = pd.DataFrame(rev_id_final_df['review_id'], columns = ['review_id'])

location_final_df = pd.concat([rev_id_final_df2,pre_location_df], axis=1)

    
end_time = time.time()

print("elapsed time:\n--- %s seconds ---" % (end_time - start_time))
print("--- %s minutes ---" % ((end_time - start_time)/60))
print("--- %s hours ---" % ((end_time - start_time)/3600))

In [4]:
final_df_0 = pd.merge(df_result_final, all_ratings_final_df, how="left", on=['review_id'])
final_df = pd.merge(final_df_0, location_final_df, how="left", on=['review_id'])

In [75]:
final_df.to_csv('output_glassdoor_Subway_213_216.csv', index=False, sep=';')